<a href="https://colab.research.google.com/github/waleligntewabe/Automatic-Question-Generation/blob/main/Final_Sentence_Parsing_based_AQG_using_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy
import re

# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

# Define a function to analyze the sentence
def analyze_sentence(sentence):
    doc = nlp(sentence)

    nsubj, dobj, pobj, prep, npadvmod, advmod,s = [],[], [], [], [], [], []

    adverb_types = {
    "Degree": ["Somewhat", "Enormously", "Totally", "Utterly", "Perfectly", "Absolutely", "Practically", "Barely",
               "Terribly", "Pretty", "Thoroughly", "Rather", "Extremely", "Incredibly", "Most", "Purely", "Little",
               "Virtually", "Very", "Fully", "Indeed", "Hardly", "Almost", "Strongly", "So", "Less", "Fairly",
               "Greatly", "Entirely", "Highly", "Lots", "Really", "Quite", "Too", "Completely", "Well", "Far",
               "Deeply", "Badly", "Scarcely", "Positively", "How", "Enough", "Awfully"],
    "Manner": ["Promptly", "Powerfully", "Calmly", "Politely", "Equally", "Faithfully", "Generously", "Gladly",
               "Joyously", "Easily", "Beautifully", "Kindly", "Bravely", "Poorly", "Shyly", "Foolishly", "Selfishly",
               "Angrily", "Awkwardly", "Violently", "Suspiciously", "Inadequately", "Irritably", "Hungrily",
               "Generally", "Frequently", "Noisily"],
    "Place": ["Behind", "About", "Upstairs", "Up", "Out", "Far", "Inside", "Right", "There", "Backward/s", "Nearby",
              "On", "Away", "Back", "Indoors", "Overseas", "Abroad", "Down", "Over", "Towards", "Here", "Anywhere",
              "Underground", "Elsewhere", "Off", "Regularly", "Previously", "Seldom", "Now", "Since", "Finally",
              "Tomorrow", "Everywhere"],
    "Time": ["Today", "Tonigh", "Occasionally", "Constantly", "Rarely", "Quarterly", "Weekly", "yesterday", "Later",
             "Then", "Eventually", "Suddenly", "Mysteriously", "Infrequently", "Continually", "Yearly", "Monthly",
             "Fortnightly", "First", "Annually", "Yearly", "Tonight", "Tomorrow", "Recently", "Late", "Accidentally",
             "Seriouly", "Deliberately", "Never", "Seriously"],
    "Frequency": ["Exactly", "Intermittently", "Quarterly", "Weekly", "Solemnly"]
    }

    for token in doc:
        s.append(token.text)
        if token.dep_ == 'nsubj':
            nsubj = s
            s = []
            subjNer = nlp(token.text)
            for ent in subjNer.ents:
                print("Subject type=", ent.label_, ent.text)

        if token.dep_ == 'ROOT':
            dobj = s
            s = []
            subjNer = nlp(token.text)
            for ent in subjNer.ents:
                print("Verb type=", ent.label_, ent.text)

        if token.dep_ in ['pobj', 'dobj', 'acomp']:
            pobj = s
            s = []
            subjNer = nlp(token.text)
            for ent in subjNer.ents:
                print("Object type=", ent.label_, ent.text)

        if token.dep_ == 'prep':
            prep = s
            s = []
            subjNer = nlp(token.text)
            for ent in subjNer.ents:
                print("Preposition type=", ent.label_, ent.text)

        if token.dep_ == 'npadvmod':
            npadvmod = s
            s = []

        if token.dep_ == 'advmod':
            advmod = s
            s = []
        # ... (implement other conditions similarly)

        # Categorize adverbs (keep this logic as it is)
        # Categorize adverbs based on adverb type dataset
    adverb_type_category = ""
    for adverb in advmod or npadvmod:
        adverb = re.sub("[']", "", adverb)
        adverb = re.sub("[,]", "", adverb)
        adverb = re.sub("[[]", "", adverb)
        adverb = re.sub("[]]", "", adverb)
        adverb = re.sub("[)]", "", adverb)
        adverb = re.sub("[(]", "", adverb)
        #print("adverb=",adverb)
        if adverb in adverb_types["Degree"]:
            adverb_type_category = "Degree"
            degree_adverb=adverb
        elif adverb in adverb_types["Manner"]:
            adverb_type_category = "Manner"
            manner_adverb=adverb
        elif adverb in adverb_types["Place"]:
            adverb_type_category = "Place"
            place_adverb=adverb
        elif adverb in adverb_types["Time"]:
            adverb_type_category = "Time"
            #print("time_adverb=",adverb)
            time_adverb=adverb
        elif adverb in adverb_types["Frequency"]:
            adverb_type_category = "Frequency"
            frequency_adverb=adverb

    # Print the results
    print("Subject=", nsubj)
    print("Verb=", dobj)
    print("Object=", pobj)
    #print("Preposition=", prep)
    print("Time Adverb=", time_adverb)
    #print("Place Adverb=", place_adverb)
    #print("Degree Adverb=", degree_adverb)
    #print("Frequency Adverb=", frequency_adverb)
    #print("Manner Adverb=", manner_adverb)
    return (nsubj,dobj,pobj,time_adverb)

In [ ]:
!pip install -q gradio

In [ ]:
import gradio as gr
import spacy
import re

# Load the Spacy model
nlp = spacy.load("en_core_web_sm")

# Function to analyze a sentence and extract its components
def analyze_sentence(sentence):
    doc = nlp(sentence)
    components = {
        'nsubj': [], 'verb': [], 'obj': [], 'prep': [], 'time_adverb': [], 'advmod':[],'npadvmod':[],
        'place_adverb': [], 'degree_adverb': [],  'manner_adverb': [], 'frequency_adverb': []
    }

    adverb_types = {
      "Degree": ["Somewhat", "Enormously", "Totally", "Utterly", "Perfectly", "Absolutely", "Practically", "Barely",
                "Terribly", "Pretty", "Thoroughly", "Rather", "Extremely", "Incredibly", "Most", "Purely", "Little",
                "Virtually", "Very", "Fully", "Indeed", "Hardly", "Almost", "Strongly", "So", "Less", "Fairly",
                "Greatly", "Entirely", "Highly", "Lots", "Really", "Quite", "Too", "Completely", "Well", "Far",
                "Deeply", "Badly", "Scarcely", "Positively", "How", "Enough", "Awfully"],
      "Manner": ["Promptly", "Powerfully", "Calmly", "Politely", "Equally", "Faithfully", "Generously", "Gladly",
                "Joyously", "Easily", "Beautifully", "Kindly", "Bravely", "Poorly", "Shyly", "Foolishly", "Selfishly",
                "Angrily", "Awkwardly", "Violently", "Suspiciously", "Inadequately", "Irritably", "Hungrily",
                "Generally", "Frequently", "Noisily"],
      "Place": ["Behind", "About", "Upstairs", "Up", "Out", "Far", "Inside", "Right", "There", "Backward/s", "Nearby",
                "On", "Away", "Back", "Indoors", "Overseas", "Abroad", "Down", "Over", "Towards", "Here", "Anywhere",
                "Underground", "Elsewhere", "Off", "Regularly", "Previously", "Seldom", "Now", "Since", "Finally",
                "Tomorrow", "Everywhere"],
      "Time": ["Today", "Tonigh", "Occasionally", "Constantly", "Rarely", "Quarterly", "Weekly", "yesterday", "Later",
              "Then", "Eventually", "Suddenly", "Mysteriously", "Infrequently", "Continually", "Yearly", "Monthly",
              "Fortnightly", "First", "Annually", "Yearly", "Tonight", "Tomorrow", "Recently", "Late", "Accidentally",
              "Seriouly", "Deliberately", "Never", "Seriously"],
      "Frequency": ["Exactly", "Intermittently", "Quarterly", "Weekly", "Solemnly"]
      }
# Extract components
    for token in doc:

        if token.dep_ == 'nsubj':
            components['nsubj'].append(token.text)
        if token.dep_ == 'ROOT':
            components['verb'].append(token.text)
        if token.dep_ in ['pobj', 'dobj', 'acomp']:
            components['obj'].append(token.text)
        if token.dep_ == 'prep':
            components['prep'].append(token.text)

        if token.dep_ == 'npadvmod' or 'advmod':
            adverb = token.text

            if adverb in adverb_types["Degree"]:
                adverb_type_category = "Degree"
                degree_adverb=adverb
                components['degree_adverb'].append(token.text)
            elif adverb in adverb_types["Manner"]:
                adverb_type_category = "Manner"
                manner_adverb=adverb
                components['manner_adverb'].append(token.text)
            elif adverb in adverb_types["Place"]:
                adverb_type_category = "Place"
                place_adverb=adverb
                components['place_adverb'].append(token.text)
            elif adverb in adverb_types["Time"]:
                adverb_type_category = "Time"
                print("time_adverb=",adverb)
                time_adverb=adverb
                components['time_adverb'].append(token.text)
                print("Time=",components['time_adverb'])
            elif adverb in adverb_types["Frequency"]:
                adverb_type_category = "Frequency"
                frequency_adverb=adverb
                components['frequency_adverb'].append(token.text)

    return components

# Function to generate questions
def generate_questions(subject, verb, obj, prep, time_adverb):
    questions = []

    # Generate Wh-questions
    if subject:
        questions.append(f"Who {verb} {obj} {prep} {time_adverb}?")

    if obj:
        questions.append(f"What did {subject} {verb} {prep} {time_adverb}?")

    if time_adverb:
        questions.append(f"When did {subject} {verb} {obj} {prep}?")

    # Generate Yes/No questions
    if subject and verb and obj:
        questions.append(f"Did {subject} {verb} {obj}?")

    return questions

# Function to generate and display questions
def display_questions(components):
    questions = generate_questions(
        subject=components['nsubj'],
        verb=components['verb'],
        obj=components['obj'],
        prep=components['prep'],
        time_adverb=components['time_adverb']
    )
        #place_adverb=components['place_adverb'],
        #degree_adverb=components['degree_adverb'],
        #manner_adverb=components['manner_adverb'],
        #frequency_adverb=components['frequency_adverb']




In [ ]:
# Old Modified function to work with Gradio
def display_questions_with_gradio(sentence):
    components = analyze_sentence(sentence)
    print(components)
    questions=display_questions(components)
    print(questions)
    # Generating questions

    # Generating questions
    #questions = generate_questions(subject, verb, obj, prep, time_adverb, place_adverb,degree_adverb, manner_adverb, frequency_adverb)


    # Format the questions for display
    #formatted_questions = "\n".join([f"Question {i}: {q}" for i, q in enumerate(questions, 1)])
    #return formatted_questions

    # Displaying generated questions
    for i, question in enumerate(questions, 1):
        question = re.sub("[']", "", question)
        question = re.sub("[,]", "", question)
        question = re.sub("[[]", "", question)
        question = re.sub("[]]", "", question)
        question = re.sub("[)]", "", question)
        question = re.sub("[(]", "", question)
        #print(f"Question {i}: {question}")
        formatted_questions = "\n".join([f"Question {i}: {question}"])
    return formatted_questions

In [ ]:
#New
def display_questions_with_gradio(sentence):
    components = analyze_sentence(sentence)
    questions = generate_questions(
        components['nsubj'],
        components['verb'],
        components['obj'],
        components['prep'],
        components['time_adverb']
    )

    # Check if questions list is not None and not empty
    if questions:
        formatted_questions = "\n".join([f"Question {i}: {q}" for i, q in enumerate(questions, 1)])

        formatted_questions = re.sub("[']", "", formatted_questions)
        formatted_questions = re.sub("[,]", "", formatted_questions)
        formatted_questions = re.sub("[[]", "", formatted_questions)
        formatted_questions = re.sub("[]]", "", formatted_questions)
        formatted_questions = re.sub("[)]", "", formatted_questions)
        formatted_questions = re.sub("[(]", "", formatted_questions)

    else:
        formatted_questions = "No questions generated."

    return formatted_questions


In [ ]:
analyze_sentence("John ate an apple yesterday")

time_adverb= yesterday
Time= ['yesterday']


{'nsubj': ['John'],
 'verb': ['ate'],
 'obj': ['apple'],
 'prep': [],
 'time_adverb': ['yesterday'],
 'advmod': [],
 'npadvmod': [],
 'place_adverb': [],
 'degree_adverb': [],
 'manner_adverb': [],
 'frequency_adverb': []}

In [ ]:
display_questions_with_gradio("John ate an apple yesterday")

time_adverb= yesterday
Time= ['yesterday']


'Question 1: Who ate apple  yesterday?\nQuestion 2: What did John ate  yesterday?\nQuestion 3: When did John ate apple ?\nQuestion 4: Did John ate apple?'

In [ ]:
# Set up Gradio interface
iface = gr.Interface(
    fn=display_questions_with_gradio,
    inputs="text",
    outputs="text"
)

# Launch the interface
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0819d303c9bbb2d1ab.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# **Other**

In [ ]:
import spacy
import re

nlp = spacy.load("en_core_web_sm")

doc = nlp('John ate an apple yesterday')

s = []
nsubj = []
dobj = []
pobj = []
prep = []
advmod=[]
npadvmod=[]
time_adverb = []
place_adverb = []
manner_adverb = []
degree_adverb = []
frequency_adverb = []

adverb_types = {
    "Degree": ["Somewhat", "Enormously", "Totally", "Utterly", "Perfectly", "Absolutely", "Practically", "Barely",
               "Terribly", "Pretty", "Thoroughly", "Rather", "Extremely", "Incredibly", "Most", "Purely", "Little",
               "Virtually", "Very", "Fully", "Indeed", "Hardly", "Almost", "Strongly", "So", "Less", "Fairly",
               "Greatly", "Entirely", "Highly", "Lots", "Really", "Quite", "Too", "Completely", "Well", "Far",
               "Deeply", "Badly", "Scarcely", "Positively", "How", "Enough", "Awfully"],
    "Manner": ["Promptly", "Powerfully", "Calmly", "Politely", "Equally", "Faithfully", "Generously", "Gladly",
               "Joyously", "Easily", "Beautifully", "Kindly", "Bravely", "Poorly", "Shyly", "Foolishly", "Selfishly",
               "Angrily", "Awkwardly", "Violently", "Suspiciously", "Inadequately", "Irritably", "Hungrily",
               "Generally", "Frequently", "Noisily"],
    "Place": ["Behind", "About", "Upstairs", "Up", "Out", "Far", "Inside", "Right", "There", "Backward/s", "Nearby",
              "On", "Away", "Back", "Indoors", "Overseas", "Abroad", "Down", "Over", "Towards", "Here", "Anywhere",
              "Underground", "Elsewhere", "Off", "Regularly", "Previously", "Seldom", "Now", "Since", "Finally",
              "Tomorrow", "Everywhere"],
    "Time": ["Today", "Tonigh", "Occasionally", "Constantly", "Rarely", "Quarterly", "Weekly", "yesterday", "Later",
             "Then", "Eventually", "Suddenly", "Mysteriously", "Infrequently", "Continually", "Yearly", "Monthly",
             "Fortnightly", "First", "Annually", "Yearly", "Tonight", "Tomorrow", "Recently", "Late", "Accidentally",
             "Seriouly", "Deliberately", "Never", "Seriously"],
    "Frequency": ["Exactly", "Intermittently", "Quarterly", "Weekly", "Solemnly"]
}

for token in doc:
    s.append(token.text)
    if token.dep_ == 'nsubj':
        nsubj = s
        s = []
        subjNer = nlp(token.text)
        for ent in subjNer.ents:
            print("Subject type=", ent.label_, ent.text)

    if token.dep_ == 'ROOT':
        dobj = s
        s = []
        subjNer = nlp(token.text)
        for ent in subjNer.ents:
            print("Verb type=", ent.label_, ent.text)

    if token.dep_ in ['pobj', 'dobj', 'acomp']:
        pobj = s
        s = []
        subjNer = nlp(token.text)
        for ent in subjNer.ents:
            print("Object type=", ent.label_, ent.text)

    if token.dep_ == 'prep':
        prep = s
        s = []
        subjNer = nlp(token.text)
        for ent in subjNer.ents:
            print("Preposition type=", ent.label_, ent.text)

    if token.dep_ == 'npadvmod':
        npadvmod = s
        s = []

    if token.dep_ == 'advmod':
        advmod = s
        s = []

# Categorize adverbs based on adverb type dataset
adverb_type_category = ""
for adverb in advmod or npadvmod:
    adverb = re.sub("[']", "", adverb)
    adverb = re.sub("[,]", "", adverb)
    adverb = re.sub("[[]", "", adverb)
    adverb = re.sub("[]]", "", adverb)
    adverb = re.sub("[)]", "", adverb)
    adverb = re.sub("[(]", "", adverb)
    #print("adverb=",adverb)
    if adverb in adverb_types["Degree"]:
        adverb_type_category = "Degree"
        degree_adverb=adverb
    elif adverb in adverb_types["Manner"]:
        adverb_type_category = "Manner"
        manner_adverb=adverb
    elif adverb in adverb_types["Place"]:
        adverb_type_category = "Place"
        place_adverb=adverb
    elif adverb in adverb_types["Time"]:
        adverb_type_category = "Time"
        #print("time_adverb=",adverb)
        time_adverb=adverb
    elif adverb in adverb_types["Frequency"]:
        adverb_type_category = "Frequency"
        frequency_adverb=adverb

# Print the results
print("Subject=", nsubj)
print("Verb=", dobj)
print("Object=", pobj)
#print("Preposition=", prep)
print("Time Adverb=", time_adverb)
#print("Place Adverb=", place_adverb)
#print("Degree Adverb=", degree_adverb)
#print("Frequency Adverb=", frequency_adverb)
#print("Manner Adverb=", manner_adverb)

Subject type= PERSON John
Object type= ORG apple
Subject= ['John']
Verb= ['ate']
Object= ['an', 'apple']
Time Adverb= yesterday


In [ ]:
# Automatic Question Generation based on Extracted Information

# Function to generate questions
def generate_questions1(subject, verb, obj, prep, time_adverb, place_adverb, degree_adverb, manner_adverb, frequency_adverb):
    questions = []

    # Generate Wh-questions
    if subject:
        questions.append(f"Who {verb} {obj} {prep} {time_adverb} {place_adverb} {degree_adverb} {manner_adverb} {frequency_adverb}?")

    if obj:
        questions.append(f"What did {subject} {verb} {prep} {time_adverb} {place_adverb} {degree_adverb} {manner_adverb} {frequency_adverb}?")

    if time_adverb:
        questions.append(f"When did {subject} {verb} {obj} {prep} {place_adverb} {degree_adverb} {manner_adverb} {frequency_adverb}?")

    if place_adverb:
        questions.append(f"Where did {subject} {verb} {obj} {prep} {time_adverb} {degree_adverb} {manner_adverb} {frequency_adverb}?")

    if degree_adverb:
        questions.append(f"How {degree_adverb} did {subject} {verb} {obj} {prep} {time_adverb} {place_adverb} {manner_adverb} {frequency_adverb}?")

    if manner_adverb:
        questions.append(f"How did {subject} {verb} {obj} {prep} {time_adverb} {place_adverb} {degree_adverb} {frequency_adverb}?")

    if frequency_adverb:
        questions.append(f"How often did {subject} {verb} {obj} {prep} {time_adverb} {place_adverb} {degree_adverb} {manner_adverb}?")

    # Generate Yes/No questions
    if subject and verb and obj:
        questions.append(f"Did {subject} {verb} {obj} {prep} {time_adverb} {place_adverb} {degree_adverb} {manner_adverb} {frequency_adverb}?")

    return questions

# Using extracted information
subject = nsubj
verb = dobj
obj = pobj
prep = prep
time_adverb = time_adverb
place_adverb = place_adverb
degree_adverb = degree_adverb
manner_adverb = manner_adverb
frequency_adverb = frequency_adverb

# Generating questions
questions = generate_questions1(subject, verb, obj, prep, time_adverb, place_adverb, degree_adverb, manner_adverb, frequency_adverb)

# Displaying generated questions
for i, question in enumerate(questions, 1):
    question = re.sub("[']", "", question)
    question = re.sub("[,]", "", question)
    question = re.sub("[[]", "", question)
    question = re.sub("[]]", "", question)
    question = re.sub("[)]", "", question)
    question = re.sub("[(]", "", question)
    print(f"Question {i}: {question}")


Question 1: Who ate an apple  yesterday    ?
Question 2: What did John ate  yesterday    ?
Question 3: When did John ate an apple     ?
Question 4: Did John ate an apple  yesterday    ?


In [ ]:
# Automatic Question Generation based on Extracted Information

# Function to generate questions
def generate_questions(subject, verb, obj, prep, time_adverb, place_adverb):
    questions = []

    # Generate Wh-questions
    if subject:
        questions.append(f"Who {verb} {obj} {prep} {time_adverb} {place_adverb}?")

    if obj:
        questions.append(f"What did {subject} {verb} {prep} {time_adverb} {place_adverb}?")

    if time_adverb:
        questions.append(f"When did {subject} {verb} {obj} {prep} {place_adverb}?")

    if place_adverb:
        questions.append(f"Where did {subject} {verb} {obj} {prep} {time_adverb}?")

    # Generate Yes/No questions
    if subject and verb and obj:
        questions.append(f"Did {subject} {verb} {obj}?")

    return questions

# Using extracted information
subject = nsubj
verb = dobj
obj = pobj
prep = prep
time_adverb = time_adverb
place_adverb = place_adverb

# Generating questions
questions = generate_questions(subject, verb, obj, prep, time_adverb, place_adverb)

# Displaying generated questions
for i, question in enumerate(questions, 1):
    question = re.sub("[']", "", question)
    question = re.sub("[,]", "", question)
    question = re.sub("[[]", "", question)
    question = re.sub("[]]", "", question)
    question = re.sub("[)]", "", question)
    question = re.sub("[(]", "", question)
    print(f"Question {i}: {question}")

Question 1: Who ate an apple  yesterday ?
Question 2: What did John ate  yesterday ?
Question 3: When did John ate an apple  ?
Question 4: Did John ate an apple?


# **GUI**

In [ ]:
def display_questions(questions):
  # Displaying generated questions
  for i, question in enumerate(questions, 1):
      question = re.sub("[']", "", question)
      question = re.sub("[,]", "", question)
      question = re.sub("[[]", "", question)
      question = re.sub("[]]", "", question)
      question = re.sub("[)]", "", question)
      question = re.sub("[(]", "", question)
      print(f"Question {i}: {question}")
      return question

In [ ]:
# Example list of questions
questions_list = ["What's your name?", "How are you?"]

# Calling the function
display_questions(questions_list)


Question 1: Whats your name?


'Whats your name?'

In [ ]:
def greet(name):
  return "Hello " + Input Text + "!"

greet("World")

In [ ]:
!pip install -q gradio

In [ ]:
import gradio

gradio.Interface(greet, "text", "text").launch(share=True)

# **GUI**

In [ ]:
!pip install gradio

In [ ]:
!pip install --upgrade typing_extensions

In [ ]:
!pip install --upgrade gradio

In [ ]:
!pip install --upgrade fastapi starlette

In [ ]:
!pip check

In [ ]:
import gradio as gr

def greet(name):
    return "Hello " + name + "!"

demo = gr.Interface(fn=greet, inputs="text", outputs="text")
demo.launch()

In [ ]:
import spacy
import re
import gradio as gr

nlp = spacy.load("en_core_web_sm")

def process_text(text):
    s = []
    nsubj = []
    dobj = []
    pobj = []
    prep = []
    time_adverb = []
    place_adverb = []
    degree_adverb = []
    manner_adverb = []
    frequency_adverb = []

    adverb_types = {
        "Degree": ["Somewhat", "Enormously", "Totally", "Utterly", "Perfectly", "Absolutely", "Practically", "Barely",
                   "Terribly", "Pretty", "Thoroughly", "Rather", "Extremely", "Incredibly", "Most", "Purely", "Little",
                   "Virtually", "Very", "Fully", "Indeed", "Hardly", "Almost", "Strongly", "So", "Less", "Fairly",
                   "Greatly", "Entirely", "Highly", "Lots", "Really", "Quite", "Too", "Completely", "Well", "Far",
                   "Deeply", "Badly", "Scarcely", "Positively", "How", "Enough", "Awfully"],
        "Manner": ["Promptly", "Powerfully", "Calmly", "Politely", "Equally", "Faithfully", "Generously", "Gladly",
                   "Joyously", "Easily", "Beautifully", "Kindly", "Bravely", "Poorly", "Shyly", "Foolishly", "Selfishly",
                   "Angrily", "Awkwardly", "Violently", "Suspiciously", "Inadequately", "Irritably", "Hungrily",
                   "Generally", "Frequently", "Noisily"],
        "Place": ["Behind", "About", "Upstairs", "Up", "Out", "Far", "Inside", "Right", "There", "Backward/s", "Nearby",
                  "On", "Away", "Back", "Indoors", "Overseas", "Abroad", "Down", "Over", "Towards", "Here", "Anywhere",
                  "Underground", "Elsewhere", "Off", "Regularly", "Previously", "Seldom", "Now", "Since", "Finally",
                  "Tomorrow", "Everywhere"],
        "Time": ["Today", "Tonigh", "Occasionally", "Constantly", "Rarely", "Quarterly", "Weekly", "Yesterday", "Later",
                 "Then", "Eventually", "Suddenly", "Mysteriously", "Infrequently", "Continually", "Yearly", "Monthly",
                 "Fortnightly", "First", "Annually", "Yearly", "Tonight", "Tomorrow", "Recently", "Late", "Accidentally",
                 "Seriouly", "Deliberately", "Never", "Seriously", "NaN"],
        "Frequency": ["Exactly", "Intermittently", "Quarterly", "Weekly", "Solemnly"]
    }

    doc = nlp(text)

    for token in doc:
        s.append(token.text)
        if token.dep_ == 'nsubj':
            nsubj = s
            s = []
            subjNer = nlp(token.text)
            for ent in subjNer.ents:
                print("Subject type=", ent.label_, ent.text)

        if token.dep_ == 'ROOT':
            dobj = s
            s = []
            subjNer = nlp(token.text)
            for ent in subjNer.ents:
                print("Verb type=", ent.label_, ent.text)

        if token.dep_ in ['pobj', 'dobj', 'acomp']:
            pobj = s
            s = []
            subjNer = nlp(token.text)
            for ent in subjNer.ents:
                print("Object type=", ent.label_, ent.text)

        if token.dep_ == 'prep':
            prep = s
            s = []
            subjNer = nlp(token.text)
            for ent in subjNer.ents:
                print("Preposition type=", ent.label_, ent.text)

        if token.dep_ == 'npadvmod':
            npadvmod = s
            s = []

        if token.dep_ == 'advmod':
            advmod = s
            s = []

    # Categorize adverbs based on adverb type dataset
    adverb_type_category = ""
    for adverb in advmod:
        adverb = re.sub("[']", "", adverb)
        adverb = re.sub("[,]", "", adverb)
        adverb = re.sub("[[]", "", adverb)
        adverb = re.sub("[]]", "", adverb)
        adverb = re.sub("[)]", "", adverb)
        adverb = re.sub("[(]", "", adverb)

        if adverb in adverb_types["Degree"]:
            adverb_type_category = "Degree"
            degree_adverb.append(adverb)
        elif adverb in adverb_types["Manner"]:
            adverb_type_category = "Manner"
            manner_adverb.append(adverb)
        elif adverb in adverb_types["Place"]:
            adverb_type_category = "Place"
            place_adverb.append(adverb)
        elif adverb in adverb_types["Time"]:
            adverb_type_category = "Time"
            time_adverb.append(adverb)
        elif adverb in adverb_types["Frequency"]:
            adverb_type_category = "Frequency"
            frequency_adverb.append(adverb)

    for adverb in npadvmod:
        adverb = re.sub("[']", "", adverb)
        adverb = re.sub("[,]", "", adverb)
        adverb = re.sub("[[]", "", adverb)
        adverb = re.sub("[]]", "", adverb)
        adverb = re.sub("[)]", "", adverb)
        adverb = re.sub("[(]", "", adverb)

        if adverb in adverb_types["Degree"]:
            adverb_type_category = "Degree"
            degree_adverb.append(adverb)
        elif adverb in adverb_types["Manner"]:
            adverb_type_category = "Manner"
            manner_adverb.append(adverb)
        elif adverb in adverb_types["Place"]:
            adverb_type_category = "Place"
            place_adverb.append(adverb)
        elif adverb in adverb_types["Time"]:
            adverb_type_category = "Time"
            time_adverb.append(adverb)
        elif adverb in adverb_types["Frequency"]:
            adverb_type_category = "Frequency"
            frequency_adverb.append(adverb)

    # Generating questions
    questions = []

    # Function to generate questions
    def generate_questions(subject, verb, obj, prep, time_adverb, place_adverb, degree_adverb, manner_adverb,
                           frequency_adverb):
        generated_questions = []

        # Generate Wh-questions
        if subject:
            generated_questions.append(
                f"Who {verb} {obj} {prep} {time_adverb} {place_adverb} {degree_adverb} {manner_adverb} {frequency_adverb}?")

        if obj:
            generated_questions.append(
                f"What did {subject} {verb} {prep} {time_adverb} {place_adverb} {degree_adverb} {manner_adverb} {frequency_adverb}?")

        if time_adverb:
            generated_questions.append(
                f"When did {subject} {verb} {obj} {prep} {place_adverb} {degree_adverb} {manner_adverb} {frequency_adverb}?")

        if place_adverb:
            generated_questions.append(
                f"Where did {subject} {verb} {obj} {prep} {time_adverb} {degree_adverb} {manner_adverb} {frequency_adverb}?")

        if degree_adverb:
            generated_questions.append(
                f"How {degree_adverb[0]} did {subject} {verb} {obj} {prep} {time_adverb} {place_adverb} {manner_adverb} {frequency_adverb}?")

        if manner_adverb:
            generated_questions.append(
                f"How did {subject} {verb} {obj} {prep} {time_adverb} {place_adverb} {degree_adverb} {frequency_adverb}?")

        if frequency_adverb:
            generated_questions.append(
                f"How often did {subject} {verb} {obj} {prep} {time_adverb} {place_adverb} {degree_adverb} {manner_adverb}?")

        # Generate Yes/No questions
        if subject and verb and obj:
            generated_questions.append(
                f"Did {subject} {verb} {obj} {prep} {time_adverb} {place_adverb} {degree_adverb} {manner_adverb} {frequency_adverb}?")

        return generated_questions

    # Using extracted information
    subject = nsubj
    verb = dobj
    obj = pobj
    prep = prep
    time_adverb = time_adverb
    place_adverb = place_adverb
    degree_adverb = degree_adverb
    manner_adverb = manner_adverb
    frequency_adverb = frequency_adverb

    # Generating questions
    questions = generate_questions(subject, verb, obj, prep, time_adverb, place_adverb, degree_adverb, manner_adverb,
                                   frequency_adverb)

    return {"input_text": text, "questions": questions}


iface = gr.Interface(fn=process_text, inputs="text", outputs=["text", "text"])
iface.launch()


ImportError: ignored